In [1]:
from azureml.core import Workspace, Experiment, Environment
ws = Workspace.from_config()
#ws = Workspace.get(name="quick-starts-ws-169190", subscription_id= "cdbe0b43-92a0-4715-838a-f2648cc7ad21" , resource_group="aml-quickstarts-169190")
exp = Experiment(workspace=ws, name="UdacityWE")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-169231
Azure region: southcentralus
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-169231


In [2]:
%%writefile conda_dependencies.yml
dependencies:
  - python=3.6.2
  - scikit-learn
  - numpy
  - pandas
  - pip:
    - azureml-defaults

Overwriting conda_dependencies.yml


In [3]:
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
#cpu_cluster.delete()

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.core import ScriptRunConfig

# Specify parameter sampler
ps = RandomParameterSampling({"C":uniform(0,1)}) ### YOUR CODE HERE ###

# Specify a Policy
policy = None ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py

est = ScriptRunConfig(source_directory=".", script='train.py',compute_target=cpu_cluster,environment=sklearn_env) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                             hyperparameter_sampling=ps,
                             policy=None,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=4,
                             max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
best_run1 = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics1 = best_run1.get_metrics()
parameter_values = best_run1.get_details()['runDefinition']['arguments']
print('\n Accuracy:', best_run_metrics1['Accuracy'])


 Accuracy: 0.9107189973614775


In [17]:
import joblib
# Get your best run and save the model from that run.
C1 =float(parameter_values[1])
C2=str(int(C1*10000))
print('Best run model is saved in model'+C2+'.joblib')
### YOUR CODE HERE ###
import os 
os.system("python train.py --C "+str(C1))

Best run model is saved in model9448.joblib


0

In [18]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Datastore, Dataset

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
datastore_paths = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = Dataset.Tabular.from_delimited_files(path=datastore_paths)  ### YOUR CODE HERE ###
    
    

In [19]:
from train import clean_data
import numpy as np
import pandas as pd

# Use the clean_data function to clean your data.
#x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(ds)
alltot= np.arange(len(x))
train= np.random.choice(alltot,size=2630, replace=False)
test= np.delete(alltot, train,0)
x_train = x.iloc[train]
y_train = y[train]
x_test= x.iloc[test]
y_test = y[test]

In [20]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

training_data=pd.concat([x_train,y_train], axis=1)
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= "accuracy",
    training_data=training_data,
    label_column_name= "y",
    n_cross_validations= 3)



In [21]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.core.experiment import Experiment
 
automl_experiment = Experiment(ws,'automl_experiment')
automl_run = automl_experiment.submit(automl_config)
automl_run.wait_for_completion(show_output=True)

2021-12-27:01:40:35,854 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2021-12-27:01:40:35,881 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2021-12-27:01:41:06,151 INFO     [utils.py:157] NumExpr defaulting to 4 threads.
2021-12-27:01:41:06,464 WARNING  [local_experiment_driver.py:148] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_7058b05d-bee1-44db-bddf-dd28e7b3ee0e,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|307                              |1                                |2630                                  |
+---------------------------------+----------

2021-12-27:01:56:59,357 INFO     [explanation_client.py:332] Using default datastore for uploads


Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_7058b05d-bee1-44db-bddf-dd28e7b3ee0e,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:00:24       0.8943    0.8943
         1   MaxAbsScaler XGBoostClassifier                 0:00:24       0.9030    0.9030
         2   MaxAbsScaler ExtremeRandomTrees                0:00:27       0.7715    0.9030
         3   SparseNormalizer XGBoostClassifier             0:00:24       0.9030    0.9030
         4   MaxAbsScaler LightGBM                          0:00:23       0.9015    0.9030
         5   MaxAbsScaler LightGBM                          0:00:23       0.8836    0.9030
         6   StandardScalerWrapper XGBoostClassifier        0:00:23       0.8985    0.9030
         7   MaxAbsS

{'runId': 'AutoML_7058b05d-bee1-44db-bddf-dd28e7b3ee0e',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-12-27T01:41:07.40024Z',
 'endTimeUtc': '2021-12-27T01:55:18.887565Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_experiment","subscription_id":"48a74bb7-9950-4cc1-9caa-5d50f995cc55","resource_group":"aml-quickstarts-169231","workspace_name":"quick-starts-ws-169231","region":"southcentralus","compute_target":"local","spark_service":nu

In [24]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = automl_run.get_output()
print(best_run)

print(fitted_model)

Run(Experiment: automl_experiment,
Id: AutoML_7058b05d-bee1-44db-bddf-dd28e7b3ee0e_30,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=0, reg_lambda=0.10416666666666667, subsample=0.7, tree_method='auto'))], verbose=False)), ('15', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l2')), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced', criterion='gini', max_depth=None, max_features='sqrt', max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.01, min_

In [31]:
best_run.get_details()

{'runId': 'AutoML_7058b05d-bee1-44db-bddf-dd28e7b3ee0e_30',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-12-27T01:54:34.161043Z',
 'endTimeUtc': '2021-12-27T01:54:56.075073Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl_experiment\',\'compute_target\':\'local\',\'subscription_id\':\'48a74bb7-9950-4cc1-9caa-5d50f995cc55\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_7058b05d-bee1-44db-bddf-dd28e7b3ee0e_30","experiment_name":null,"workspace_name":"quick-starts-ws-169231","subscription_id":"48a74bb7-9950-4cc1-9caa-5d50f

In [32]:
best_run.get_details()['properties']['best_individual_pipeline_score']

'0.9034244839800829'